In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import redditutils as ru
import word2vecReader as wvr

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
!ls word2vec_twitter_model

README                     word2vecReaderUtils.py
__pycache__                word2vec_twitter_model.bin
word2vecReader.py


In [5]:
df = pd.read_csv('cleaned_shower.csv', index_col = 0)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
df['quality'] = df['score'].apply(lambda x: ru.make_labels(x))

In [32]:
df.quality.mean()

0.3927000364998175

## Train test split

In [57]:
df = df[df['title'].notna()]

In [59]:
X = df['title']
y = df['quality']

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)

In [61]:
X_train_short = X_train[:100000]
y_train_short = y_train[:100000]

## Word embedding with word2vec

In [22]:
model_path = "./word2vec_twitter_model.bin"
model = KeyedVectors.load_word2vec_format(model_path, binary=True, unicode_errors='ignore')

In [45]:
def make_features(words, model, num_features):
    features = np.zeros(num_features)
    
    model_vocab = set(model.index2word)
    
    num_words = 0
    
    # Loop over words in documents. If the word is in model's vocabulary,
    # generate its feature vector
    for w in words:
        if w in model_vocab:
            num_words += 1
            features = np.add(features, model[w])
            
    # Normalize the feature vector
    features = np.divide(features, num_words)
    
    return features

In [46]:
def document_vecs(docs, model, num_features):
    # Get the average feature vector for each showerthought based on the words it's comprised of
    counter = 0
    
    doc_vector = np.zeros((len(docs), num_features))
    
    for d in docs:
        if counter%1000 == 0:
            print(f'Finished document number {counter}')
            
            # Add this document's feature vector to doc_vector
            doc_vector[counter] = make_features(d, model, num_features)
            
        counter += 1
    return doc_vector

In [62]:
features = document_vecs(X_train_short, model, 400)

Finished document number 0
Finished document number 1000
Finished document number 2000
Finished document number 3000
Finished document number 4000
Finished document number 5000
Finished document number 6000
Finished document number 7000
Finished document number 8000
Finished document number 9000
Finished document number 10000
Finished document number 11000
Finished document number 12000
Finished document number 13000
Finished document number 14000
Finished document number 15000
Finished document number 16000
Finished document number 17000
Finished document number 18000
Finished document number 19000
Finished document number 20000
Finished document number 21000
Finished document number 22000
Finished document number 23000
Finished document number 24000
Finished document number 25000
Finished document number 26000
Finished document number 27000
Finished document number 28000
Finished document number 29000
Finished document number 30000
Finished document number 31000
Finished document num